# 4.3 Interpretability

This notebook reproduces the example from Section 4.3 of the paper 'Linux Kernel Configurations at Scale: A Dataset for Performance and Evolution Analysis' (EASE 2025). It uses a Random Forest model to identify influential options affecting binary size in TuxKConfig version 5.8 (OpenML ID: 46744).

## Steps:
1. **Load Dataset**: Fetch version 5.8 from OpenML.
2. **Train Model**: Fit a Random Forest regressor.
3. **Extract Importance**: Rank the top 5 options by feature importance scores.



In [ ]:
import openml
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Known targets to exclude from features (avoid data leakage)
known_targets = [
    "vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP",
    "BZIP2-bzImage", "BZIP2-vmlinux", "BZIP2",
    "LZMA-bzImage", "LZMA-vmlinux", "LZMA",
    "XZ-bzImage", "XZ-vmlinux", "XZ",
    "LZO-bzImage", "LZO-vmlinux", "LZO",
    "LZ4-bzImage", "LZ4-vmlinux", "LZ4"
]

# Step 1: Load TuxKConfig v5.8
dataset = openml.datasets.get_dataset(46744)
data, _, _, _ = dataset.get_data(dataset_format="dataframe")

# Step 2: Separate features and target
X = data.drop(columns=known_targets, errors="ignore")
y = data["vmlinux"]

# Step 3: Train Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Step 4: Extract and rank top 5 options
importances = model.feature_importances_
feature_names = X.columns
top_indices = importances.argsort()[-5:][::-1]
top_features = [(feature_names[i], importances[i]) for i in top_indices]
for feature, score in top_features:
    print(f'{feature}: {score:.4f}')